In [1]:
%matplotlib inline

In [2]:
import os,sys
import numpy as np
from matplotlib import pyplot as plt
import shapely
import logging

import workflow
import workflow.source_list
import workflow.ui
import workflow.conf
import workflow.extrude
import workflow.rowcol


workflow.ui.setup_logging(1,None)


ModuleNotFoundError: No module named 'workflow.rowcol'

In [ ]:
HUC = '060102020103'
crs = workflow.conf.default_crs()

logging.info("")
logging.info("Meshing HUC: {}".format(HUC))
logging.info("="*30)
logging.info('Target projection: "{}"'.format(crs['init']))
    

In [ ]:
sources = workflow.source_list.get_default_sources()
workflow.source_list.log_sources(sources)

In [ ]:
# collect datasets
huc, centroid = workflow.get_split_form_hucs(sources['HUC'], HUC, crs=crs, centering=False)
rivers, centroid = workflow.get_rivers_by_bounds(sources['hydrography'], 
                                                 huc.exterior().bounds, crs, HUC, centering=centroid)

args = workflow.ui.default_simplify_options()
args.simplify=50
args.prune_reach_size=4
rivers = workflow.simplify_and_prune(huc, rivers, args)
    

In [ ]:
args = workflow.ui.default_triangulate_options()
args.delaunay = False

# triangulation options:
# Refine triangles if their area (in m^2) is greater than A(d), where d is the distance from the triangle
# centroid to the nearest stream.
# A(d) is a piecewise linear function -- A = A0 if d <= d0, A = A1 if d >= d1, and linearly interpolates
# between the two endpoints.
d0 = 500; d1 = 2000
A0 = 10000; A1 = 50000
args.refine_distance = [d0,A0,d1,A1]

# make 2D mesh
mesh_points2, mesh_tris = workflow.triangulate(huc, rivers, args, diagnostics=True)

# get a raster for the elevation map
dem_profile, dem = workflow.get_raster_on_shape(sources['DEM'], huc.exterior(), crs)

# elevate the triangle nodes to the dem
mesh_points3 = workflow.elevate(mesh_points2, crs, dem, dem_profile)

In [ ]:
# plot the resulting surface mesh
fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111)
mp = workflow.plot.triangulation(mesh_points3, mesh_tris, linewidth=0.5, color='elevation')
fig.colorbar(mp, orientation="horizontal", pad=0.1)
workflow.plot.hucs(huc, 'k', linewidth=0.7)
workflow.plot.rivers(rivers, color='blue', linewidth=0.5)
ax.set_aspect('equal', 'datalim')
t = ax.set_title('2D mesh and digital elevation map')

In [ ]:
# convert the triangulation into a 2D mesh for extrusion
m2 = workflow.extrude.Mesh2D(mesh_points3, list(mesh_tris))

In [ ]:
# download the NRCS soils data as shapes and convert it to raster
import workflow.sources.manager_nrcs
import matplotlib.cm

# -- download the shapes
target_bounds = huc.exterior().bounds
soil_survey, soil_ids, centroid = workflow.get_shapes_in_bounds(sources['soil type'], huc.exterior().bounds, crs)
soil_ids = np.array(soil_ids, np.int32)

target_filename = os.path.join(workflow.conf.rcParams['data dir'], 'soil_survey', 
                       'soil_survey_W{0:.0g}_N{3:.0g}.tif')

# -- create a map from sane IDs to map unit ids from NRCS
#soil_indices = dict((1001+i, v) for (i,v) in enumerate(soil_properties))
#soil_ids_new = range(1001,1001+len(soil_ids))

# -- color a raster by the polygons (this makes identifying a triangle's value much more efficient)
soil_color, soil_profile, img_bounds = workflow.color_raster_from_shapes(target_bounds, 10, target_filename, 
                                                                soil_survey, soil_ids, crs)

geologic_type = 101 * np.ones((len(mesh_tris),),'i')

In [ ]:
# plot the soil survey
extent = [img_bounds[0], img_bounds[2], img_bounds[1], img_bounds[3]]

soil_color = np.where(soil_color == -1, np.nan, soil_color)
current_cmap = matplotlib.cm.get_cmap()
current_cmap.set_bad(color='white')

vmax = max(soil_ids)
vmin = min(soil_ids)
plt.imshow(soil_color, vmax=vmax, vmin=vmin, extent=extent)
plt.colorbar()

workflow.plot.hucs(huc, color='r')

plt.show()

In [ ]:
# pick the soil type for triangle centroids
tri_centroids = m2.centroids()
tri_soil_ids = soil_color[workflow.rowcol.rowcol(soil_profile['transform'], tri_centroids[:,0], tri_centroids[:,1])]

soil_labels = dict()
soil_label_counter = 1001

fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111)
mp = workflow.plot.triangulation(mesh_points3, mesh_tris, linewidth=0.5, color=tri_soil_ids)
plt.show()

In [ ]:
# layer extrusion
# -- data structures needed for extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []
z = 0.0

# -- soil layer --
#  top 6 m
#  5 cm initial top cell
#  10 cells
#  expanding dz, growing with depth
ncells = 9
dz = 0.05
layer_dz = 4

def telescope_factor(ncells, dz, layer_dz):
    """Calculates a telescoping factor"""
    if ncells * dz > layer_dz:
        raise ValueError(("Cannot telescope {} cells of thickness at least {} "+
                          "and reach a layer of thickness {}").format(ncells, dz, layer_dz))

    import scipy.optimize
    def seq(r):
        calc_layer_dz = dz * (1 - r**ncells)/(1-r)
        #print('tried: {} got: {}'.format(r, calc_layer_dz))
        return layer_dz - calc_layer_dz
    res = scipy.optimize.root_scalar(seq, x0=1.0001, x1=2)
    return res.root

tele = telescope_factor(ncells, dz, layer_dz)
logging.info("Got telescoping factor: {}".format(tele))
for i in range(ncells):
    layer_types.append('constant')
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(tri_soil_ids)
    z += dz
    dz *= tele
    
# one more 2m layer makes 6m
dz = 2.0
layer_types.append('constant')
layer_data.append(dz)
layer_ncells.append(1)
layer_mat_ids.append(tri_soil_ids)
z += dz

# -- geologic layer --
# keep going for 2m cells until we hit the bottom of
# the domain
layer_types.append("constant")
layer_data.append(40 - z) # depth of bottom of domain is 40 m
layer_ncells.append(int(round(layer_data[-1] / dz)))
layer_mat_ids.append(geologic_type)

# print the summary
workflow.extrude.Mesh3D.summarize_extrusion(layer_types, layer_data, layer_ncells, layer_mat_ids)

In [ ]:
# extrude
m3 = workflow.extrude.Mesh3D.extruded_Mesh2D(m2, layer_types, layer_data, layer_ncells, layer_mat_ids)

In [ ]:
# save to disk
m3.write_exodus('mesh_{}.exo'.format(HUC))

In [ ]:
# layer extrusion
# -- data structures needed for extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []
z = 0.0

# -- soil layer --
#  top 6 m
#  5 cm initial top cell
#  10 cells
#  expanding dz, growing with depth
ncells = 9
dz = 0.05
layer_dz = 4

def telescope_factor(ncells, dz, layer_dz):
    """Calculates a telescoping factor"""
    if ncells * dz > layer_dz:
        raise ValueError(("Cannot telescope {} cells of thickness at least {} "+
                          "and reach a layer of thickness {}").format(ncells, dz, layer_dz))

    import scipy.optimize
    def seq(r):
        calc_layer_dz = dz * (1 - r**ncells)/(1-r)
        #print('tried: {} got: {}'.format(r, calc_layer_dz))
        return layer_dz - calc_layer_dz
    res = scipy.optimize.root_scalar(seq, x0=1.0001, x1=2)
    return res.root

tele = telescope_factor(ncells, dz, layer_dz)
logging.info("Got telescoping factor: {}".format(tele))
for i in range(ncells):
    layer_types.append('constant')
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(tri_soil_ids)
    z += dz
    dz *= tele
    
# one more 2m layer makes 6m
dz = 2.0
layer_types.append('constant')
layer_data.append(dz)
layer_ncells.append(1)
layer_mat_ids.append(tri_soil_ids)
z += dz

# -- geologic layer --
# keep going for 2m cells until we hit the bottom of
# the domain
layer_types.append("constant")
layer_data.append(40 - z) # depth of bottom of domain is 40 m
layer_ncells.append(int(round(layer_data[-1] / dz)))
layer_mat_ids.append(geologic_type)

# print the summary
workflow.extrude.Mesh3D.summarize_extrusion(layer_types, layer_data, layer_ncells, layer_mat_ids)